## Testando metodos para reduzir a dimensionalidade com pipelines

1. Carregar o conjunto de dados iris, e algumas bibliotecas para reduzir dimensões.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.decomposition import PCA, NMF, TruncatedSVD
from sklearn.manifold import Isomap
%matplotlib inline

## Como fazer

1. instanciar objeto pipeline com duas partes
    -  um objeto para reduzir a dimensionalidade
    -  um estimador com método de previsão

In [6]:
pipe = Pipeline([
    ("reduce_dim", PCA()),
    ("classify", SVC())
])

2. Isomap vêm do módulo manifold e o algoritmo NMF(non-negative matrix factorization) utiliza o SVD para dividir a matriz en fatores não negativos, o propósito principal é comparar o desempenho com outros algoritmos, mas este é mais usado em NLP(natural language processing) onde a factorização não pode ser negativo.

In [4]:
param_grid = [
    {
        "reduce_dim":[PCA(), NMF(), Isomap(), TruncatedSVD()],
        "reduce_dim__n_components": [2,3],
        "classify": [SVC(), LinearSVC()],
        "classify__C": [1,10,100,1000]
    }
]

3. Por para funciona

In [8]:
grade = GridSearchCV(pipe, cv=3, n_jobs=-1, param_grid=param_grid)
iris = load_iris()
grade.fit(iris.data, iris.target)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:383: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/opt/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_isomap.py:383: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/opt/anaconda3/lib/python3.11/site-packages/scipy/sparse/_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self.

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('reduce_dim', PCA()),
                                       ('classify', SVC())]),
             n_jobs=-1,
             param_grid=[{'classify': [SVC(), LinearSVC()],
                          'classify__C': [1, 10, 100, 1000],
                          'reduce_dim': [PCA(), NMF(), Isomap(),
                                         TruncatedSVD()],
                          'reduce_dim__n_components': [2, 3]}])

4. olhar pelo melhor parâmetro para escolher como melhor modelo

In [9]:
grade.best_params_

{'classify': SVC(),
 'classify__C': 100,
 'reduce_dim': NMF(),
 'reduce_dim__n_components': 3}

In [10]:
grade.best_score_

0.98

5. para criar uma estrutura de resultado dos dados, seguinte comando

In [11]:
import pandas as pd
resultados_DF = pd.DataFrame(grade.cv_results_)

In [13]:
resultados_DF.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classify,param_classify__C,param_reduce_dim,param_reduce_dim__n_components,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000963,0.000008,0.000482,0.000176,SVC(),1,PCA(),2,"{'classify': SVC(), 'classify__C': 1, 'reduce_...",0.96,0.94,0.96,0.953333,0.009428,30
1,0.001052,0.000093,0.000381,0.000017,SVC(),1,PCA(),3,"{'classify': SVC(), 'classify__C': 1, 'reduce_...",0.98,0.98,0.94,0.966667,0.018856,9
2,0.004466,0.003142,0.000776,0.000457,SVC(),1,NMF(),2,"{'classify': SVC(), 'classify__C': 1, 'reduce_...",0.98,0.96,0.96,0.966667,0.009428,9


6. finalmente pode preveer numa instância com  metodo para teste

In [14]:
iris_X = iris.data
X_test = iris_X[1::2]

In [19]:
grade.predict(X_test)[:5]

array([0, 0, 0, 0, 0])

In [22]:
iris_X[1:11:2]

array([[4.9, 3. , 1.4, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [5. , 3.4, 1.5, 0.2],
       [4.9, 3.1, 1.5, 0.1]])